In [1]:
import pyetrade

consumer_key = "a694bf19a1a7ee1724e35a21e626bffe"
consumer_secret = "c52c35186ec76bcbe37e308e2e37d942e179dc07921efb3c79094b525785dd25"

oauth = pyetrade.ETradeOAuth(consumer_key, consumer_secret)
print(oauth.get_request_token())  # Use the printed URL

verifier_code = input("Enter verification code: ")
tokens = oauth.get_access_token(verifier_code)

print("\nCopy the following 4 values into the manager's arguments:")
print(consumer_key)
print(consumer_secret)
print(tokens["oauth_token"])
print(tokens["oauth_token_secret"])

https://us.etrade.com/e/t/etws/authorize?key=a694bf19a1a7ee1724e35a21e626bffe&token=KuWubcVo2QdGGfbtTFRhKyjBca0rRxgq8FYtq5/yI5k=


Enter verification code:  TIGQW



Copy the following 4 values into the manager's arguments:
a694bf19a1a7ee1724e35a21e626bffe
c52c35186ec76bcbe37e308e2e37d942e179dc07921efb3c79094b525785dd25
zdMeKEtIxKHNBF08zKLxJPGidBnFycppbbrU/3FUkeM=
OQEFpRzNOtdiYgJSZGO8XcgDJtistmwDz5mRkXw4WsA=


In [2]:
import options_manager
import logging
import datetime

logging.basicConfig(level=logging.INFO)

manager = options_manager.OptionsManager(
    consumer_key="a694bf19a1a7ee1724e35a21e626bffe", 
    consumer_secret="c52c35186ec76bcbe37e308e2e37d942e179dc07921efb3c79094b525785dd25", 
    oauth_token="zdMeKEtIxKHNBF08zKLxJPGidBnFycppbbrU/3FUkeM=",
    oauth_secret="OQEFpRzNOtdiYgJSZGO8XcgDJtistmwDz5mRkXw4WsA="
)

In [3]:
manager.get_market_data("GOOG")

MarketData(ticker='GOOG', company_name='ALPHABET INC CAP STK CL C', market_price=2765.485, high_52=3037.0, low_52=1699.0, beta=1.16, next_earnings_date='01/31/2022')

In [4]:
manager.get_options_info(
    ticker="LVS", 
    min_strike=30,
    max_strike=20, # all arguments below have defaults and don't need to be passed
    increment=100,
    month_look_ahead=3,
    min_volume=0,
    min_open_interest=0,
    min_annualized_return=0.0,
    contracts_to_buy=1, # defaults to max available
)

INFO:options_manager:Found 0 valid options for ticker LVS.


[]

In [5]:
import pathlib
import pandas as pd
import threading
from multiprocessing.dummy import Pool as ThreadPool 


from pprint import pprint

sector_path = pathlib.Path("./data/sectors.csv")
sector_df = pd.read_csv(sector_path)
sector_df.fillna("",inplace=True)

options_df = pd.DataFrame()

skip = ["NVR", "KSU"]


thread_pool = ThreadPool(5)

tickers = [ticker for ticker in sector_df["Ticker"].unique() if ticker not in skip]
tickers = tickers[:50]

def helper(ticker):
    options_info = manager.get_options_info(
        ticker=ticker, 
        min_strike=30,
        max_strike=20, # all arguments below have defaults and don't need to be passed
        increment=100,
        month_look_ahead=3,
        min_volume=0,
        min_open_interest=0,
        min_annualized_return=0.0,
        contracts_to_buy=1, # defaults to max available
    )

    data = [{key: value for key, value in option.items() if not isinstance(value, dict)} for option in options_info]

    return pd.DataFrame(data)


    
results = thread_pool.map(helper, tickers)

pd.concat(results).reset_index()

    

INFO:options_manager:Found 0 valid options for ticker CMCSA.
INFO:options_manager:Found 0 valid options for ticker LYV.
INFO:options_manager:Found 0 valid options for ticker VIAC.
INFO:options_manager:Found 0 valid options for ticker F.
INFO:options_manager:Found 0 valid options for ticker TMUS.
INFO:options_manager:Found 0 valid options for ticker NWS.
INFO:options_manager:Found 0 valid options for ticker T.
INFO:options_manager:Found 0 valid options for ticker VZ.
INFO:options_manager:Found 0 valid options for ticker GM.
INFO:options_manager:Found 3 valid options for ticker CHTR.
INFO:options_manager:Found 0 valid options for ticker LUMN.
INFO:options_manager:Found 0 valid options for ticker SIRI.
INFO:options_manager:Found 0 valid options for ticker MGM.
INFO:options_manager:Found 0 valid options for ticker LCID.
INFO:options_manager:Found 0 valid options for ticker DISCK.
INFO:options_manager:Found 0 valid options for ticker PENN.
INFO:options_manager:Found 0 valid options for tick

,index,optionType,strikePrice,symbol,bid,ask,volume,openInterest,netChange,lastPrice,expiryDate,marketPrice,belowMarketPct
0,0,PUT,500.0,CHTR,0.0,3.8,0,878,0.0,0.4,2022-01-21,627.895,20.4
1,1,PUT,500.0,CHTR,0.9,4.8,0,0,0.0,0.0,2022-02-18,627.895,20.4
2,2,PUT,500.0,CHTR,2.75,3.2,0,236,0.0,2.1,2022-03-18,627.895,20.4
3,0,PUT,800.0,TSLA,1.9,2.17,2401,10752,0.4,2.15,2022-01-21,1076.220,25.7
4,1,PUT,800.0,TSLA,12.8,13.35,2258,14975,1.85,13.4,2022-02-18,1076.220,25.7
5,2,PUT,800.0,TSLA,23.75,24.85,202,3956,2.91,24.71,2022-03-18,1076.220,25.7
6,0,PUT,400.0,DPZ,0.1,4.3,0,304,0.0,0.2,2022-01-21,532.205,24.8
7,1,PUT,400.0,DPZ,0.3,3.2,0,0,0.0,0.0,2022-02-18,532.205,24.8
8,2,PUT,400.0,DPZ,1.2,3.2,1,150,0.19,2.04,2022-03-18,532.205,24.8
9,0,PUT,200.0,MCD,0.03,0.07,0,3273,0.0,0.04,2022-01-21,270.855,26.2
